In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# %pdb on

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]
# NPBのMGの初期変数
mg_size :list[int] = [32, 64, 128, 256, 512]
mg_nit: list[int] = [4, 10, 20, 35, 50]

train_mg_process :list[int] = [4, 8, 16, 32, 64]
train_mg_size :list[int] = [4, 8, 16, 32, 64]
train_mg_nit :list[int] = [5, 10, 15, 20, 25]

test_mg_process :list[int] = [128, 256, 512]
test_mg_size :list[int] = [128, 256, 512]
test_mg_nit :list[int] = [30, 40, 50]

# NPBのEPの初期変数
train_ep_process :list[int] = [4, 8, 16, 32, 64]
train_ep_size :list[str] = ["S", "W", "A", "B", "C"]
test_ep_process :list[int] = [128, 256, 512]
test_ep_size :list[str] = ["D", "E", "F"]

# NPBのFTの初期変数
train_ft_process :list[int] = [2, 4, 8, 16, 32, 64]

# NPBのISの初期変数
train_is_process :list[int] = [2, 4, 8, 16, 32, 64]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

list_csvDir_mg = [
    "./csv_files/mg_1st/",
    "./csv_files/mg_2nd/",
    "./csv_files/mg_3rd/",
]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# 必要な結果

* 関数コール回数予測
    * モデル適合度
    * モデル予測精度
* 実行時間予測
    * モデル適合度
    * モデル予測精度

# 必要なブツ

* モデル構築用データ
* 予測対象用データ
* 関数コール回数の予測モデル
* 関数の総実行時間の予測モデル
* 1コール当たりの関数の総実行時間の予測モデル


In [2]:
class ContentsForExtraP:
    """class ContentsForExtraP
    
    ExtraPによるモデルの生成

    生成されるモデルは次の通り。

    * 関数コール回数の予測モデル
    * 関数の総実行時間の予測モデル
    * 1コール当たりの関数の総実行時間の予測モデル
    
    """

    def __init__(self, trainDF :pd.DataFrame, testDF :pd.DataFrame,expVars :list[str], resVars :list[str], resVarsPerCall :list[str]):
        """__init__()

        初期化変数

        Args:
            trainDF(pd.DataFrame):モデル構築用データ
            testDF(pd.DataFrame):予測対象用データ
            expVars(list[str]):説明変数を格納したリスト
            resVars(list[str]):目的変数を格納したリスト
            resVarsPerCall(list[str]):1コール当たりの目的変数を格納したリスト
        
        """
        self.trainDF = trainDF
        self.testDF = testDF
        self.expVars = expVars
        self.resVars = resVars
        self.resVarsPerCall = resVarsPerCall

    def _build_model(self, resVar :str):
        """_build_model()

        引数から指定した条件でモデルを構築する。

        Args:
            resVar(str):目的変数
        
        """

        pass

    def build_all_models(self):
        """build_all_models()

        モデルを構築する関数。目的変数はself.resVars, self.resVarsPerCallを利用。

        """

        pass

    def return_models(self):
        """return_models()
        
        構築した各種モデルを返す関数。
        
        """

        pass
